In [2]:
import pandas as pd
import math
import numpy as np
from tqdm import tqdm
from collections import defaultdict

def triple_barrier(price, ub, lb, max_period):

    def end_price(s):
        return np.append(s[(s / s[0] > ub) | (s / s[0] < lb)], s[-1])[0]/s[0]
    
    r = np.array(range(max_period))
    
    def end_time(s):
        return np.append(r[(s / s[0] > ub) | (s / s[0] < lb)], max_period-1)[0]

    p = price.rolling(max_period).apply(end_price, raw=True).shift(-max_period+1)
    t = price.rolling(max_period).apply(end_time, raw=True).shift(-max_period+1)
    t = pd.Series([t.index[int(k+i)] if not math.isnan(k+i) else np.datetime64('NaT') 
                   for i, k in enumerate(t)], index=t.index).dropna()

    signal = pd.Series(1, p.index)
    signal.loc[p > ub] = 2
    signal.loc[p < lb] = 0
    signal[-(max_period-1):] = np.nan
    ret = pd.DataFrame({'triple_barrier_profit':p, 'triple_barrier_sell_time':t, 'triple_barrier_signal':signal})

    return ret
    

In [3]:
close_df = pd.read_csv('./data/indicator_data/close.csv')

with open('codes_300.txt', 'r') as f:
    codes = f.read().split()

code_para_map = defaultdict(lambda: defaultdict(list))

for code in tqdm(codes): 
    close_serie = close_df[code].rename('close')

    for d in [10, 20, 30, 40, 60, 120]:
        up_threshold = 1.01
        down_threshold = 0.99
        label_name = str(d)+'d'
        label_serie = triple_barrier(close_serie, up_threshold, down_threshold, d + 1)['triple_barrier_signal'].rename(label_name)

        df = pd.concat([close_serie, label_serie], axis=1)
        df = df.dropna()
        df = df[83:]

        while len(df[label_name].value_counts()) < 3 or df[label_name].value_counts()[2] > df[label_name].value_counts()[1]:
            up_threshold += 0.001
            down_threshold -= 0.001
            label_serie = triple_barrier(close_serie, up_threshold, down_threshold, d + 1)['triple_barrier_signal'].rename(label_name)

            df = pd.concat([close_serie, label_serie], axis=1)

            df = df.dropna()
            df = df[83:]
        code_para_map[code][label_name] += [up_threshold, down_threshold]
        

100%|██████████| 355/355 [3:56:30<00:00, 39.97s/it]  


In [4]:
# import json
# with open('trend_params.json') as jsonfile:
#     code_para_map = json.load(jsonfile)


for code in code_para_map:
    for d in code_para_map[code]:
        code_para_map[code][d][0] = round(code_para_map[code][d][0], 3)
        code_para_map[code][d][1] = round(code_para_map[code][d][1], 3)


In [8]:
len(code_para_map)

355

In [6]:
import json
    
with open("trend_params_300.json", "w") as outfile:
    json.dump(code_para_map, outfile)